In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

### Importation des bibliothèques ###
import math
import os
import sys
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import trackpy as tp
import functions_analyze as lib
from scipy.signal import find_peaks

# Définition de l'appareil (CPU ou GPU) pour les opérations PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device for torch operations:", device)

def force_field_inbox(coordinates_diff, distances, Req, R0, Frep, Fadh,
                      coeff_a=None, coeff_rep=None):
    """
    Calcule le champ de forces entre les cellules en prenant en compte les forces
    de répulsion et d'adhésion, selon les distances entre les cellules.

    Paramètres:
    - coordinates_diff: Tensor des différences de coordonnées entre les cellules
    - distances: Tensor des distances entre les cellules
    - Req: Rayon d'équilibre (μm)
    - R0: Rayon d'interaction maximale (μm)
    - Frep: Intensité de la force de répulsion
    - Fadh: Intensité de la force d'adhésion
    - coeff_a: Coefficient pour la force quadratique (non utilisé ici)
    - coeff_rep: Coefficient pour ajuster la force de répulsion
    """
    Rlim = 0.000001  # Limite inférieure pour éviter la division par zéro
    R = torch.norm(coordinates_diff, dim=2)  # Calcul des distances entre les cellules
    # Empêche la force de répulsion de diverger en fixant une valeur minimale pour R
    R = torch.where(R > Rlim * torch.ones(1, device=device),
                    R, Rlim * torch.ones(1, device=device)).to(device)
    
    # Calcul de la force d'adhésion (linéaire entre Req et R0)
    force = torch.where(torch.logical_and(R < R0, R > Req),
                        -((Fadh / (R0 - Req)) * R - Fadh * Req / (R0 - Req)),
                        torch.zeros_like(R)).to(device)
    
    # Calcul de la force de répulsion (proportionnelle à 1/R)
    force = torch.where(R <= Req,
                        -Frep * coeff_rep * (1 / Req - 1 / R), force).to(device)
    
    # Calcul du champ de forces résultant en sommant les contributions des autres cellules
    # On normalise les vecteurs de différence pour obtenir les directions
    force_field = torch.sum(force[:, :, None] *
                            torch.nn.functional.normalize(coordinates_diff, dim=2), axis=1)
    
    return force_field

def plot_environment(cells, camp_field, space_size, path_saving=None, iteration=None):
    """
    Trace l'environnement de simulation, y compris les positions des cellules et
    la concentration du champ de cAMP.

    Paramètres:
    - cells: Liste des cellules
    - camp_field: Objet représentant le champ de cAMP
    - space_size: Taille de l'espace de simulation
    - path_saving: Chemin pour sauvegarder l'image générée
    - iteration: Numéro de l'itération (utilisé pour le nom du fichier)
    """
    fig, axis = plt.subplots(figsize=(6, 6))
    plt.xlim(0, space_size)
    plt.ylim(0, space_size)

    # Traçage de la carte de concentration du cAMP
    extent = [0, space_size, 0, space_size]
    cmap = plt.cm.viridis  # Palette de couleurs pour la concentration
    im = axis.imshow(camp_field.signal.T, origin='lower', extent=extent, cmap=cmap, alpha=0.5)
    plt.colorbar(im, ax=axis, label='Concentration de cAMP')

    # Récupération des positions des cellules
    x = [cell.position[0].item() for cell in cells]
    y = [cell.position[1].item() for cell in cells]

    # Attribution des couleurs en fonction de la population
    colors = ['blue' if cell.pop == 'Population 1' else 'red' for cell in cells]

    # Tracé des cellules sur le graphique
    axis.scatter(x, y, s=5, color=colors, alpha=0.1, edgecolors='k')

    plt.xlabel('Position X (micromètres)')
    plt.ylabel('Position Y (micromètres)')
    plt.title(f'Temps: {iteration * DELTA_T:.2f} min')
    # Sauvegarde de la figure
    plt.savefig(f'{path_saving}image_{iteration}.png',
                bbox_inches='tight', dpi=300, pad_inches=0)
    plt.close()

def plot_function(pas, Req, R0, Frep, Fadh, a, coeff_rep):
    """
    Trace les fonctions de force de répulsion et d'adhésion en fonction de la distance.

    Paramètres:
    - pas: Pas d'échantillonnage pour les distances
    - Req: Rayon d'équilibre (μm)
    - R0: Rayon d'interaction maximale (μm)
    - Frep: Intensité de la force de répulsion
    - Fadh: Intensité de la force d'adhésion
    - a: Coefficient quadratique (non utilisé ici)
    - coeff_rep: Coefficient pour ajuster la force de répulsion
    """
    # Calcul des coefficients pour les forces (non utilisés ici)
    b = (Fadh - a * (R0 ** 2 - Req ** 2)) / (R0 - Req)
    c = -Req * (a * Req + (Fadh - a * (R0 ** 2 - Req ** 2)) / (R0 - Req))

    fig, axis = plt.subplots(figsize=(6, 6))
    plt.xlim(0, R0)
    plt.ylim(-Frep, Fadh)

    # Tracé de la force de répulsion pour R entre 0 et Req
    axis.plot(np.arange(pas, Req, pas), [
              Frep * coeff_rep * (1 / Req - 1 / R) for R in np.arange(pas, Req, pas)], label='Répulsion')
    # Tracé de la force d'adhésion pour R entre Req et R0
    axis.plot(np.arange(Req, R0, pas), [
              (Fadh / (R0 - Req)) * (R - Req) for R in np.arange(Req, R0, pas)], label='Adhésion')

    plt.xlabel('Distance')
    plt.ylabel('Force')
    plt.legend()
    plt.show()

class CellAgent:
    def __init__(self, id, pop, position, velocity,
                 velocity_magnitude, persistence, space_size,
                 tau, noise, c_params, sensitivity_cAMP_threshold):
        """
        Classe représentant une cellule individuelle dans la simulation.

        Paramètres:
        - id: Identifiant unique de la cellule
        - pop: Type de population (e.g., 'Population 1' ou 'Population 2')
        - position: Position initiale de la cellule (tensor)
        - velocity: Vitesse initiale de la cellule (tensor)
        - velocity_magnitude: Norme de la vitesse initiale
        - persistence: Persistance du mouvement
        - space_size: Taille de l'espace de simulation
        - tau: Temps caractéristique pour l'alignement directionnel
        - noise: Intensité du bruit dans la direction
        - c_params: Dictionnaire des paramètres pour le modèle de FitzHugh-Nagumo
        """
        self.id = id  # ID unique de la cellule
        self.pop = pop  # Type de population
        self.position_init = position.clone().to(device)  # Position initiale (pour référence)
        self.position = position.clone().to(device)  # Position actuelle
        self.velocity = velocity.clone().to(device)  # Vitesse actuelle
        self.velocity_magnitude = velocity_magnitude  # Norme de la vitesse
        self.persistence = persistence  # Persistance du mouvement
        self.space_size = space_size  # Taille de l'espace
        self.tau = tau  # Temps caractéristique pour l'alignement directionnel
        self.noise = noise  # Intensité du bruit
        self.direction = torch.nn.functional.normalize(velocity, p=2, dim=0)  # Direction normalisée
        # États A et R pour le modèle de FitzHugh-Nagumo
        self.A = torch.tensor(0.5, device=device)  # État de l'activateur A
        self.R = torch.tensor(0.5, device=device)  # État du répresseur R
        self.c_params = c_params  # Paramètres du modèle de FitzHugh-Nagumo
        # Paramètres pour la production du cAMP
        self.D = c_params['D']  # Taux de production de cAMP
        self.af = c_params['af']  # Seuil d'activation pour la production de cAMP
        self.sensitivity_threshold = sensitivity_cAMP_threshold  # Seuil de sensibilité au cAMP


    def update_state(self, signal_value, dt):
        """
        Met à jour les états internes A et R de la cellule en fonction du signal cAMP local.

        Paramètres:
        - signal_value: Valeur du signal cAMP à la position de la cellule
        - dt: Pas de temps de la simulation
        """
        # Extraction des paramètres du modèle
        a = self.c_params['a']
        Kd = self.c_params['Kd']
        gamma = self.c_params['gamma']
        c0 = self.c_params['c0']
        epsilon = self.c_params['epsilon']
        sigma = self.c_params['sigma']
        noise_flag = self.c_params.get('noise', True)

        # Calcul de l'entrée I_S en fonction du signal cAMP local
        I_S = a * torch.log1p(signal_value / Kd)

        # Mise à jour de l'état A (activateur)
        dA = (self.A - (self.A ** 3) / 3 - self.R + I_S) * dt
        if noise_flag:
            dA += sigma * math.sqrt(dt) * torch.randn((), device=device)  # Ajout de bruit
        self.A += dA

        # Mise à jour de l'état R (répresseur)
        dR = (self.A - gamma * self.R + c0) * epsilon * dt
        self.R += dR
            
class Population:
    def __init__(self, num_cells, space_size, velocity_magnitude,
                 persistence, min_distance, pop_tag, ecart_type,
                 tau, noise, c_params, sensitivity_cAMP_threshold):
        """
        Classe représentant une population de cellules.

        Paramètres:
        - num_cells: Nombre de cellules dans la population
        - space_size: Taille de l'espace de simulation
        - velocity_magnitude: Norme moyenne de la vitesse des cellules
        - persistence: Persistance du mouvement
        - min_distance: Distance minimale entre les cellules à l'initialisation
        - pop_tag: Identifiant de la population (e.g., 'Population 1')
        - ecart_type: Écart-type pour la distribution des vitesses
        - tau: Temps caractéristique pour l'alignement directionnel
        - noise: Intensité du bruit dans la direction
        - c_params: Dictionnaire des paramètres pour le modèle de FitzHugh-Nagumo
        """
        self.num_cells = num_cells
        self.space_size = space_size
        self.velocity_magnitude = velocity_magnitude
        self.persistence = persistence
        self.min_distance = min_distance
        self.pop_tag = pop_tag  # Identifiant de la population
        self.ecart_type = ecart_type  # Écart-type pour les vitesses initiales
        self.tau = tau
        self.noise = noise
        self.c_params = c_params
        self.sensitivity_cAMP_threshold = sensitivity_cAMP_threshold
        self.cells = []
        self.initialize_cells()

    def initialize_cells(self):
        """
        Initialise les cellules de la population en leur attribuant des positions
        et des vitesses initiales, tout en respectant la distance minimale entre elles.
        """
        global cell_id_counter  # Compteur global pour les identifiants de cellules
        # Génération de positions aléatoires dans l'espace
        positions = torch.rand((self.num_cells, 2), device=device) * self.space_size
        # Génération de directions aléatoires normalisées
        directions = torch.nn.functional.normalize(torch.empty_like(positions).uniform_(-1, 1), dim=1)
        # Génération de vitesses suivant une distribution normale
        vitesses_gaussiennes = torch.normal(mean=self.velocity_magnitude, std=self.ecart_type, size=(self.num_cells,), device=device)

        # Logique pour placer les cellules en respectant la distance minimale
        if self.min_distance != 0:
            grid_size = int(np.ceil(self.space_size / self.min_distance))
            grid = [[[] for _ in range(grid_size)] for _ in range(grid_size)]
            
            for i, position in enumerate(positions):
                # Essayer de placer la cellule sans conflits
                placed = False
                while not placed:
                    grid_x = int(position[0] / self.min_distance)
                    grid_y = int(position[1] / self.min_distance)
                    # Vérification des conflits avec les cellules voisines
                    conflicts = any(torch.norm(position - other_position) < self.min_distance for other_position in grid[grid_x][grid_y])
                    if not conflicts:
                        for dx in [-1, 0, 1]:
                            for dy in [-1, 0, 1]:
                                if dx == 0 and dy == 0:
                                    continue
                                nx, ny = grid_x + dx, grid_y + dy
                                if 0 <= nx < grid_size and 0 <= ny < grid_size:
                                    conflicts = any(torch.norm(position - other_position) < self.min_distance for other_position in grid[nx][ny])
                                    if conflicts:
                                        break
                            if conflicts:
                                break
                    if not conflicts:
                        # Placement réussi sans conflit
                        grid[grid_x][grid_y].append(position)
                        velocity = directions[i] * vitesses_gaussiennes[i]
                        self.cells.append(CellAgent(cell_id_counter, self.pop_tag, position, velocity,
                                                    vitesses_gaussiennes[i], self.persistence, self.space_size,
                                                    self.tau, self.noise, self.c_params, self.sensitivity_cAMP_threshold))
                        cell_id_counter += 1
                        placed = True
                    else:
                        # Générer une nouvelle position aléatoire
                        position = torch.rand(2, device=device) * self.space_size
        else:
            # Si la distance minimale est zéro, on place les cellules sans contrainte
            for i, position in enumerate(positions):
                velocity = directions[i] * vitesses_gaussiennes[i]
                self.cells.append(CellAgent(cell_id_counter, self.pop_tag, position, velocity,
                                            vitesses_gaussiennes[i], self.persistence, self.space_size,
                                            self.tau, self.noise, self.c_params, self.sensitivity_cAMP_threshold))
                cell_id_counter += 1

class Surface:
    def get_friction(self, position):
        """
        Retourne la friction de la surface à une position donnée.
        Ici, elle est générée aléatoirement entre 0 et 0.2.

        Paramètres:
        - position: Position pour laquelle on veut la friction (non utilisée)

        Retourne:
        - friction: Valeur de friction à la position donnée
        """
        friction = torch.empty(1).uniform_(0, 0.2).to(device)
        return friction.item()
        
class cAMP:
    def __init__(self, space_size, grid_resolution, D, aPDE, initial_condition=None):
        """
        Classe représentant le champ de concentration du cAMP dans l'espace.

        Paramètres:
        - space_size: Taille de l'espace de simulation
        - grid_resolution: Résolution de la grille (taille d'une cellule de la grille)
        - D: Coefficient de diffusion du cAMP
        - aPDE: Taux de dégradation du cAMP
        - initial_condition: Condition initiale de la concentration de cAMP (optionnelle)
        """
        self.space_size = space_size
        self.grid_resolution = grid_resolution
        self.grid_size = int(space_size / grid_resolution)
        self.D = D  # Coefficient de diffusion
        self.aPDE = aPDE  # Taux de dégradation
        self.dx = grid_resolution  # Pas spatial
        self.dt = DELTA_T # self.dx ** 2 / (8 * self.D)  # Pas de temps pour la stabilité numérique

        # Création de la grille de coordonnées
        x = torch.linspace(0, space_size, self.grid_size, device=device)
        y = torch.linspace(0, space_size, self.grid_size, device=device)
        self.X, self.Y = torch.meshgrid(x, y, indexing='ij')

        # Initialisation de la grille de concentration du cAMP
        if initial_condition is None:
            # Par défaut, concentration nulle partout
            self.signal = torch.zeros((self.grid_size, self.grid_size), device=device)
        else:
            # Si une fonction ou un tableau est fourni pour l'état initial
            if callable(initial_condition):
                # Utiliser la fonction pour définir la concentration initiale
                self.signal = initial_condition(self.X, self.Y)
            elif isinstance(initial_condition, torch.Tensor):
                # Vérifier que la taille correspond
                if initial_condition.shape == (self.grid_size, self.grid_size):
                    self.signal = initial_condition.to(device)
                else:
                    raise ValueError("La forme de initial_condition ne correspond pas à la grille.")
            else:
                # Supposer que c'est une valeur scalaire
                self.signal = torch.full((self.grid_size, self.grid_size), initial_condition, device=device)
    
    def compute_laplacian(self, S):
        """
        Calcule le Laplacien du signal S avec des conditions aux limites de Neumann (flux nul).

        Paramètres:
        - S: Grille de concentration du cAMP

        Retourne:
        - laplacian_S: Grille du Laplacien de S
        """
        laplacian_S = torch.zeros_like(S)
        laplacian_S[1:-1, 1:-1] = (
            S[2:, 1:-1] + S[:-2, 1:-1] + S[1:-1, 2:] + S[1:-1, :-2] - 4 * S[1:-1, 1:-1]
        ) / self.dx ** 2
        return laplacian_S
    
    def update(self, cells):
        """
        Met à jour la concentration du cAMP en fonction de la production par les cellules,
        de la diffusion et de la dégradation.

        Paramètres:
        - cells: Liste des cellules dans la simulation
        """
        # Production de cAMP par les cellules
        A_grid = torch.zeros_like(self.signal)
        for cell in cells:
            x_idx = int(cell.position[0].item() / self.grid_resolution)
            y_idx = int(cell.position[1].item() / self.grid_resolution)
            if 0 <= x_idx < self.grid_size and 0 <= y_idx < self.grid_size:
                # La production dépend de l'état d'activation de la cellule (A)
                # On suppose que la production est proportionnelle à Heaviside(A - af)
                af = cell.af  # Seuil d'activation pour la production de cAMP
                if cell.A > af:
                    A_grid[x_idx, y_idx] += cell.D  # D est le taux de production de cAMP par les cellules activées

        # Diffusion et dégradation du cAMP
        laplacian_S = self.compute_laplacian(self.signal)
        self.signal += self.dt * (self.D * laplacian_S - self.aPDE * self.signal + A_grid)
        self.signal = torch.clamp(self.signal, min=0)  # Pour éviter les valeurs négatives
    
    def get_signal_at_position(self, position):
        """
        Renvoie la concentration du cAMP à une position donnée.

        Paramètres:
        - position: Position pour laquelle on veut connaître la concentration de cAMP

        Retourne:
        - Concentration de cAMP à la position donnée
        """
        x_idx = int(position[0].item() / self.grid_resolution)
        y_idx = int(position[1].item() / self.grid_resolution)
        if 0 <= x_idx < self.grid_size and 0 <= y_idx < self.grid_size:
            return self.signal[x_idx, y_idx]
        else:
            return torch.tensor(0.0, device=device)
            
def autovel(dX, n, tau, noise, dt, persistence):
    """
    Calcule la nouvelle direction des cellules basée sur leur déplacement,
    leur direction précédente, le bruit, le temps caractéristique tau, et la persistance.

    Paramètres:
    - dX: Déplacement de la cellule
    - n: Vecteur directionnel précédent de la cellule
    - tau: Temps caractéristique pour l'alignement directionnel
    - noise: Intensité du bruit angulaire
    - dt: Pas de temps de la simulation
    - persistence: Persistance du mouvement (non utilisé ici)

    Retourne:
    - new_direction: Nouvelle direction normalisée de la cellule
    """    
    # Normalise le vecteur de déplacement
    dX_norm = torch.nn.functional.normalize(dX, dim=1) * 0.9999999
    if persistence == 1:
        persistence = 0.9999999
    # Calcule l'angle actuel du déplacement
    theta = torch.atan2(dX_norm[:, 1], dX_norm[:, 0]).to(device)
    
    # Calcule le changement d'angle basé sur le vecteur d'orientation précédent
    dtheta = torch.arcsin((n[:, 0] * dX_norm[:, 1] - n[:, 1] * dX_norm[:, 0])) * dt / tau

    # Génère du bruit aléatoire pour la perturbation de l'angle
    rnd = (2 * math.pi * (torch.rand(len(dX), 1, device=device) - 0.5)) * noise * np.sqrt(dt)
    
    # Mise à jour de l'angle
    theta_update = theta + dtheta + rnd.squeeze(1)

    # Calcule la nouvelle direction à partir de l'angle mis à jour
    new_dir_x = torch.cos(theta_update)
    new_dir_y = torch.sin(theta_update)
    
    # Crée un nouveau tenseur pour la direction mise à jour
    new_direction = torch.stack((new_dir_x, new_dir_y), dim=1)
    
    return new_direction

# def compute_local_gradient(signal_grid, position, grid_resolution):
#     """
#     Calcule le gradient local du cAMP à la position donnée.

#     Paramètres:
#     - signal_grid: Grille de concentration du cAMP
#     - position: Position de la cellule
#     - grid_resolution: Résolution de la grille (taille d'une cellule de la grille)

#     Retourne:
#     - grad: Vecteur gradient du cAMP à la position de la cellule
#     """
    # # x_idx = int(position[0].item() / grid_resolution)
    # # y_idx = int(position[1].item() / grid_resolution)
    # # grid_size = signal_grid.shape[0]

    # # # Gestion des bords de la grille
    # # if 1 <= x_idx < grid_size - 1 and 1 <= y_idx < grid_size - 1:
    # #     # Calcul des dérivées partielles centrales
    # #     grad_x = (signal_grid[x_idx + 1, y_idx] - signal_grid[x_idx - 1, y_idx]) / (2 * grid_resolution)
    # #     grad_y = (signal_grid[x_idx, y_idx + 1] - signal_grid[x_idx, y_idx - 1]) / (2 * grid_resolution)
    # #     grad = torch.tensor([grad_x, grad_y], device=device)
    # # else:
    # #     grad = torch.tensor([0.0, 0.0], device=device)  # Pas de gradient aux bords

    # # return grad
    # x_idx = int(position[0].item() / grid_resolution) % signal_grid.shape[0]
    # y_idx = int(position[1].item() / grid_resolution) % signal_grid.shape[1]
    # grid_size_x, grid_size_y = signal_grid.shape

    # # Calcul des dérivées partielles centrales avec conditions périodiques
    # grad_x = (signal_grid[(x_idx + 1) % grid_size_x, y_idx] - signal_grid[(x_idx - 1) % grid_size_x, y_idx]) / (2 * grid_resolution)
    # grad_y = (signal_grid[x_idx, (y_idx + 1) % grid_size_y] - signal_grid[x_idx, (y_idx - 1) % grid_size_y]) / (2 * grid_resolution)
    # grad = torch.tensor([grad_x, grad_y], device=device)
    # return grad

def compute_local_gradient(signal_grid, position, grid_resolution, r_sensing):
    x_idx = int(position[0].item() / grid_resolution) % signal_grid.shape[0]
    y_idx = int(position[1].item() / grid_resolution) % signal_grid.shape[1]
    grid_size_x, grid_size_y = signal_grid.shape

    # Déterminer le nombre de mailles correspondant au rayon de détection
    n_cells = int(r_sensing / grid_resolution)

    grad_x = 0.0
    grad_y = 0.0
    count = 0

    # Parcourir les positions dans la zone de détection
    for dx in range(-n_cells, n_cells + 1):
        for dy in range(-n_cells, n_cells + 1):
            distance = math.hypot(dx * grid_resolution, dy * grid_resolution)
            if 0 < distance <= r_sensing:
                x_neighbor = (x_idx + dx) % grid_size_x
                y_neighbor = (y_idx + dy) % grid_size_y

                # Calcul de la différence de concentration
                delta_c = signal_grid[x_neighbor, y_neighbor] - signal_grid[x_idx, y_idx]

                # Calcul du vecteur directionnel vers le voisin
                unit_vector = torch.tensor([dx * grid_resolution, dy * grid_resolution], device=device) / distance

                # Ajout de la contribution au gradient
                grad_x += delta_c * unit_vector[0]
                grad_y += delta_c * unit_vector[1]
                count += 1

    if count > 0:
        grad_x /= count
        grad_y /= count

    grad = torch.tensor([grad_x, grad_y], device=device)
    return grad


PLOT = True

# In[Simulation parameters]
# Paramètres de l'espace
SPACE_SIZE = 50  # Taille de l'espace de simulation en micromètres

# Paramètres temporels
TIME_SIMU = 1000  # Temps total de simulation en minutes
DELTA_T = 0.01  # Pas de temps de la simulation en minutes
PLOT_INTERVAL = 100  # Intervalle pour la sauvegarde des images

# Paramètres de simulation
MU = 1  # Mobilité (μm / min·kg⁻¹)
F_REP = 40  # Intensité de la force de répulsion
F_ADH = 7  # Intensité de la force d'adhésion
R_EQ = 1.1  # Rayon d'équilibre (μm)
R_0 = 1.6  # Rayon maximal d'interaction (μm)
MIN_DISTANCE_INIT = R_EQ  # Distance minimale entre cellules à l'initialisation
COEFF_CARRE = 50  # Coefficient quadratique (non utilisé ici)
COEFF_REP = 0.5  # Coefficient pour ajuster la force de répulsion
# Trace la fonction de force pour visualiser les forces de répulsion et d'adhésion
plot_function(pas=0.01, Req=R_EQ, R0=R_0, Frep=F_REP,
              Fadh=F_ADH, a=COEFF_CARRE, coeff_rep=COEFF_REP)

# Facteur de fluctuation pour la vitesse
FLUCTUATION_FACTOR = 3

# Définition des cellules
PACKING_FRACTION = 0.04  # Fraction de l'espace occupée par les cellules
# Calcul du nombre de cellules en fonction de la fraction de remplissage
N_CELLS = int((PACKING_FRACTION * SPACE_SIZE ** 2) /
              (math.pi * ((R_EQ / 2) ** 2)))  # Nombre total de cellules
print(N_CELLS, "cells")
# Paramètres de la Population 1
velocity_magnitude_pop1 = 0*3  # Vitesse moyenne des cellules de la population 1 (μm/min)
ECART_TYPE_POP1 = 0.3  # Écart-type de la vitesse pour la population 1
NOISE_POP_1 = 0*8  # Intensité du bruit pour la population 1
TAU_POP_1 = 5  # Temps caractéristique pour la population 1
PERSISTENCE_POP1 = 0  # Persistance du mouvement pour la population 1
SENSITIVITY_cAMP_THRESHOLD_POP1 = 0.1
# Paramètres de la Population 2
velocity_magnitude_pop2 = 0*8  # Vitesse moyenne des cellules de la population 2 (μm/min)
ECART_TYPE_POP2 = 0.5  # Écart-type de la vitesse pour la population 2
NOISE_POP_2 = 0*5  # Intensité du bruit pour la population 2
TAU_POP_2 = 5  # Temps caractéristique pour la population 2
PERSISTENCE_POP2 = 0  # Persistance du mouvement pour la population 2
SENSITIVITY_cAMP_THRESHOLD_POP2 = 0.1

R_SENSING_GRAD = 10.0 # Distance en um sur laquelle la cellule peut sentir le gradient


# Paramètres des cellules pour le modèle de FitzHugh-Nagumo
cell_params = {
    'c0': -1.0,        # Terme constant dans l'équation de R
    'a': 5.0,        # Couplage entre le signal et l'activateur A
    'gamma': 0.1,     # Taux de relaxation du répresseur R
    'Kd': 0.1,        # Ajuster Kd pour définir la sensibilité au cAMP
    'sigma': 0.05,    # Réduire le bruit pour plus de stabilité
    'epsilon': 0.2,   # Ajuster le taux de relaxation de R
    'D': 1.0,        # Ajuster le taux de production de cAMP par les cellules
    'a0': 0.0,        # Production basale mise à zéro
    'af': 0.5,        # Seuil d'activation pour la production de cAMP
    'noise': True,
    'D_cAMP' : 20.0,  # Coefficient de diffusion du cAMP
    'aPDE' : 1.0, # 10.0  # Taux de dégradation du cAMP
    'grid_resolution' : 1.0  # Taille des cellules de la grille (μm)
}


# Définition des populations

cell_id_counter = 0  # Initialisation du compteur d'ID des cellules
population1 = Population(num_cells=int(N_CELLS / 2), space_size=SPACE_SIZE,
                         velocity_magnitude=velocity_magnitude_pop1,
                         persistence=PERSISTENCE_POP1, ecart_type=ECART_TYPE_POP1,
                         min_distance=MIN_DISTANCE_INIT, pop_tag="Population 1",
                         tau=TAU_POP_1, noise=NOISE_POP_1, c_params=cell_params,
                         sensitivity_cAMP_threshold=SENSITIVITY_cAMP_THRESHOLD_POP1)

population2 = Population(num_cells=int(N_CELLS / 2), space_size=SPACE_SIZE,
                         velocity_magnitude=velocity_magnitude_pop2,
                         persistence=PERSISTENCE_POP2, ecart_type=ECART_TYPE_POP2,
                         min_distance=MIN_DISTANCE_INIT, pop_tag="Population 2",
                         tau=TAU_POP_2, noise=NOISE_POP_2, c_params=cell_params,
                         sensitivity_cAMP_threshold=SENSITIVITY_cAMP_THRESHOLD_POP2)

# Combinaison des cellules des deux populations
cells = population1.cells + population2.cells

surface = Surface()  # Création d'une instance de Surface (non utilisée ici)

# In[Initialisation]

##### Gradient nul : valeur fixe ########
def gradient_initial_condition(X, Y):
    return torch.full_like(X, 0.5, device=device)

# # Fonction pour créer un gradient de concentration de 0 en haut à 100 en bas
# def gradient_initial_condition(X, Y):
#     return (Y / SPACE_SIZE) * 100.0  # Y varie de 0 à SPACE_SIZE

##### Gradient linéaire max au centre min en hau et bas #######
# def gradient_initial_condition(X, Y):
#     max_concentration = 25.0  # Valeur maximale de la concentration au milieu
#     return (max_concentration * (1 - ((2 * (Y - SPACE_SIZE / 2) / SPACE_SIZE) ** 2))).to(device)

##### Gradient linéaire radial partant du centre #######
# def gradient_initial_condition(X, Y):
#     X_c = SPACE_SIZE / 2
#     Y_c = SPACE_SIZE / 2
#     R_max = math.sqrt(2) * (SPACE_SIZE / 2)  # Distance maximale au coin de l'espace
#     R = torch.sqrt((X - X_c) ** 2 + (Y - Y_c) ** 2)
#     max_concentration = 10.0
#     signal = max_concentration * (1 - R / R_max)
#     signal = torch.clamp(signal, min=0.0)  # Éviter les valeurs négatives
#     return signal.to(device)

# Initialisation de la grille de cAMP
# Création du champ de cAMP avec le gradient initial
camp_field = cAMP(SPACE_SIZE,cell_params['grid_resolution'],
                  cell_params['D_cAMP'], cell_params['aPDE'],
                  initial_condition=gradient_initial_condition)


# Paramètre de chimiotaxie (sensibilité au gradient de cAMP)
chemotaxis_sensitivity = 0.5  # À ajuster selon vos besoins

positions = torch.stack([cell.position_init for cell in cells])
V0 = torch.tensor([cell.velocity_magnitude for cell in cells], device=device).unsqueeze(1)
direction = torch.stack([cell.direction for cell in cells])
positions = torch.stack([cell.position for cell in cells])

if PLOT:
    # Définition du chemin pour sauvegarder les images
    PATH = f'./simu/v1{velocity_magnitude_pop1}v2{velocity_magnitude_pop2}a{COEFF_CARRE}coefrep{COEFF_REP}fadh{F_ADH}frep{F_REP}/'
    if not os.path.exists(PATH):
        os.makedirs(PATH)
    else:
        print("WARNING : FOLDER DOES ALREADY EXIST!")
        sys.exit(0)
    # Tracé de l'environnement initial
    plot_environment(cells, camp_field, SPACE_SIZE, path_saving=PATH, iteration=0)


# In[Simulation]
time = 0  # Initialisation du temps
iteration = 1  # Compteur d'itérations
MAX_DISTANCE = np.sqrt(2 * (SPACE_SIZE / 2) ** 2)  # Distance maximale possible dans l'espace
# Création d'une liste pour stocker les données de simulation
data_list = []
while time < TIME_SIMU:
    # Mise à jour des états A et R des cellules
    for cell in cells:
        signal_value = camp_field.get_signal_at_position(cell.position)
        cell.update_state(signal_value, DELTA_T)
        if cell.A > cell.af:
            print(f"Cell {cell.id} is producing cAMP at time {time:.2f} with A = {cell.A.item():.2f}")

    # Mise à jour de la grille de cAMP
    camp_field.update(cells)

    # Calcul du gradient du cAMP pour chaque cellule et ajustement de la direction
    for cell in cells:
         # Obtenir la concentration de cAMP à la position de la cellule
        local_camp_concentration = camp_field.get_signal_at_position(cell.position)
        # Vérifier si la concentration est supérieure au seuil de sensibilité de la cellule
        if local_camp_concentration >= cell.sensitivity_threshold:
            # La cellule suit le gradient
            grad_cAMP = compute_local_gradient(camp_field.signal, cell.position, camp_field.grid_resolution, r_sensing=R_SENSING_GRAD)
            if torch.norm(grad_cAMP) > 0:
                grad_cAMP = grad_cAMP / torch.norm(grad_cAMP)  # Normalisation du gradient
                # Mise à jour de la direction de la cellule en tenant compte de la chimiotaxie
                cell.direction = (1 - chemotaxis_sensitivity) * cell.direction + chemotaxis_sensitivity * grad_cAMP
                cell.direction = torch.nn.functional.normalize(cell.direction, p=2, dim=0)
        else:
            # La cellule ne suit pas le gradient
            # Vous pouvez définir ici le comportement de la cellule lorsqu'elle ne détecte pas le cAMP
            pass  # Par exemple, ne rien changer à la direction

    # Calcul des différences de coordonnées entre les cellules (pour les forces)
    positions = torch.stack([cell.position for cell in cells])
    coordinates_diff = ((positions[:, None, :] - positions[None, :, :]))
    # Gestion des conditions périodiques aux limites (torus)
    coordinates_diff = torch.remainder(coordinates_diff - (SPACE_SIZE / 2), SPACE_SIZE) - (SPACE_SIZE / 2)
    distances = torch.stack([torch.norm(coordinates_diff[i], dim=1)
                             for i in range(0, len(coordinates_diff))])
    is_greater_than_max = torch.any(distances > MAX_DISTANCE)

    if is_greater_than_max:
        print("At least one distance is greater than the max distance.")

    # Calcul du champ de forces entre les cellules
    force_field = force_field_inbox(coordinates_diff, distances, Req=R_EQ,
                                    R0=R_0, Frep=F_REP, Fadh=F_ADH,
                                    coeff_a=COEFF_CARRE, coeff_rep=COEFF_REP)
    
    # Calcul du déplacement des cellules
    V0 = torch.tensor([cell.velocity_magnitude for cell in cells], device=device).unsqueeze(1)
    direction = torch.stack([cell.direction for cell in cells])
    # Ajout de fluctuations aléatoires à la vitesse
    fluctuations = (torch.rand(V0.shape, device=V0.device) - 0.5) * FLUCTUATION_FACTOR
    # Calcul du déplacement total : contribution du mouvement propre et des forces intercellulaires
    displacement = MU * force_field * DELTA_T + (V0 + fluctuations) * direction * DELTA_T
    positions += displacement

    # Mise à jour des positions des cellules
    for idx, cell in enumerate(cells):
        cell.position += displacement[idx]
        cell.position = torch.remainder(cell.position, SPACE_SIZE)  # Conditions périodiques aux limites
        # Mise à jour de la direction de la cellule
        new_direction = autovel(displacement[idx].unsqueeze(0), cell.direction.unsqueeze(0),
                                cell.tau, cell.noise, DELTA_T, persistence=cell.persistence)
        cell.direction = new_direction.squeeze(0)

        # Enregistrement des informations de la cellule pour l'analyse
        data_list.append({
            'frame': time,
            'particle': cell.id,
            'pop_tag': cell.pop,
            'x': cell.position[0].item(),
            'y': cell.position[1].item(),
            'dir_x': cell.direction[0].item(),
            'dir_y': cell.direction[1].item()
        })

    # Tracé des résultats à intervalles réguliers
    if PLOT:
        marker_radius = 1.1
        marker_size = (np.pi) * marker_radius ** 2
        if iteration % PLOT_INTERVAL == 0:
            plot_environment(cells, camp_field, SPACE_SIZE, path_saving=PATH, iteration=iteration)
            # Ajoutez une visualisation spécifique pour le cAMP
            plt.figure(figsize=(6, 6))
            extent = [0, SPACE_SIZE, 0, SPACE_SIZE]
            plt.imshow(camp_field.signal.cpu().numpy().T, origin='lower', extent=extent, cmap='viridis', alpha=0.8)
            plt.colorbar(label='Concentration de cAMP')
            plt.title(f'Champ de cAMP à l\'itération {iteration}')
            plt.xlabel('Position X (μm)')
            plt.ylabel('Position Y (μm)')
            plt.savefig(f'{PATH}camp_{iteration}.png', bbox_inches='tight', dpi=300, pad_inches=0)
            plt.close()


    # Mise à jour du temps et de l'itération
    time += DELTA_T
    iteration += 1
